<a href="https://colab.research.google.com/github/KwokHing/WiDS-Datathon-Patient-Survival/blob/main/WiDS_Patient_Survival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### mount google drive as local storage

from google.colab import drive
import os
drive.mount('/content/gdrive')

repo_path = '/content/gdrive/My Drive/colab/Kaggle-WiDS/'

Mounted at /content/gdrive


In [ ]:
!python 

Python 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [ ]:
### read data
trainData = os.path.join(repo_path, 'data/training_v2.csv')
testData = os.path.join(repo_path, 'data/unlabeled.csv')

train = pd.read_csv(trainData)
test = pd.read_csv(testData)

# maybe can drop gender as well here
train_v1 = train.drop(['encounter_id','patient_id','icu_id','hospital_id','readmission_status','ethnicity','gender'],axis=1)
test_v1 = test.drop(['encounter_id','patient_id','icu_id','hospital_id','readmission_status','hospital_death','ethnicity','gender'],axis=1)

print("number of rows and columns in training set is \n", train_v1.shape)
print("number of rows and columns in test set is \n", test_v1.shape)

number of rows and columns in training set is 
 (91713, 179)
number of rows and columns in test set is 
 (39308, 178)


In [ ]:
train_v1.head()

,hospital_death,age,bmi,elective_surgery,height,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,...,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,0,68.0,22.73,0,180.3,Floor,Floor,admit,CTICU,0.541667,73.9,2.3,113.0,502.01,0,0.0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0.0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0.0,14.1,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,0.05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,0,77.0,27.42,0,160.0,Floor,Floor,admit,Med-Surg ICU,0.927778,70.2,NaN,108.0,203.01,0,0.0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0.0,46.0,37.0,37.0,51.0,7.45,33.0,145.0,35.1,NaN,1.0,12.7,NaN,...,1.3,1.3,3.5,3.5,557.0,557.0,4.2,4.2,145.0,145.0,12.7,12.7,37.0,37.0,7.45,7.45,51.0,51.0,54.8,51.000000,37.0,37.0,7.45,7.45,51.0,51.0,51.0,51.0,0.47,0.29,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory
2,0,25.0,31.95,0,172.7,Emergency Department,Accident & Emergency,admit,Med-Surg ICU,0.000694,95.3,NaN,122.0,703.03,0,0.0,NaN,NaN,NaN,NaN,3.0,6.0,0.0,5.0,NaN,102.0,NaN,0.0,68.0,NaN,NaN,NaN,NaN,37.0,NaN,36.7,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
3,0,81.0,22.64,1,165.1,Operating Room,Operating Room / Recovery,admit,CTICU,0.000694,61.7,NaN,203.0,1206.03,1,0.0,NaN,NaN,NaN,0.6,4.0,6.0,0.0,5.0,185.0,114.0,25.9,1.0,60.0,30.0,30.0,142.0,7.39,4.0,NaN,34.8,NaN,1.0,8.0,62.0,...,1.6,1.1,NaN,NaN,43.0,43.0,NaN,NaN,NaN,NaN,8.8,8.8,37.0,27.0,7.44,7.34,337.0,102.0,342.5,236.666667,36.0,33.0,7.37,7.34,337.0,265.0,337.0,337.0,0.04,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
4,0,19.0,NaN,0,188.0,NaN,Accident & Emergency,admit,Med-Surg ICU,0.073611,NaN,NaN,119.0,601.01,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,0.0,103.0,NaN,NaN,NaN,NaN,16.0,NaN,36.7,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma


In [ ]:
### Imputing 'age', 'height', 'weight', 'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob'

# Initialize IterativeImputer
mice_imputer = IterativeImputer()

# Impute using fit_tranform on diabetes
train_v1[['age', 'height', 'weight', 'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob']] = mice_imputer.fit_transform(train_v1[['age', 'height', 'weight','apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob']])
test_v1[['age', 'height', 'weight', 'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob']] = mice_imputer.fit_transform(test_v1[['age', 'height', 'weight', 'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob']])

In [ ]:
def bmi(row):
  bmi = row['weight']/((row['height']/100)**2)
  return round(bmi, 2)

train_v1['bmi']=train_v1.apply(bmi,axis=1)
test_v1['bmi']=test_v1.apply(bmi,axis=1)

In [ ]:
X = train_v1.drop('hospital_death',axis = 1)
y = train_v1['hospital_death']

print(X.shape)
print(y.shape)

(91713, 178)
(91713,)


In [ ]:
# large amount of missing values

train_missing = (X.isnull().sum() / len(X)).sort_values(ascending = False)
train_missing.head()
train_missing = train_missing.index[train_missing > 0.75]
print('There are %d columns with more than 75%% missing values' % len(train_missing))
X = X.drop(columns = train_missing)
test_v1 = test_v1.drop(columns = train_missing)

There are 45 columns with more than 75% missing values


In [ ]:
X.head()

,age,bmi,elective_surgery,height,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,...,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_glucose_max,h1_glucose_min,h1_inr_max,h1_inr_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,68.0,22.73,0,180.3,Floor,Floor,admit,CTICU,0.541667,73.900000,2.3,113.0,502.01,0,0.0,0.4,31.0,2.51,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0.0,40.0,36.0,134.0,39.3,NaN,0.0,14.1,46.0,32.0,68.0,37.0,68.0,37.0,119.0,...,8.9,8.9,27.4,27.4,NaN,NaN,1.3,1.0,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100000,0.050000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,77.0,27.42,0,160.0,Floor,Floor,admit,Med-Surg ICU,0.927778,70.200000,NaN,108.0,203.01,0,0.0,NaN,9.0,0.56,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0.0,46.0,33.0,145.0,35.1,NaN,1.0,12.7,NaN,NaN,95.0,31.0,95.0,31.0,118.0,...,11.3,11.1,36.9,36.1,1.3,1.3,3.5,3.5,557.0,487.0,4.2,3.8,145.0,145.0,23.3,12.7,145.0,143.0,1.3,1.3,37.0,37.0,7.45,7.45,51.0,51.0,54.8,51.000000,0.470000,0.290000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory
2,25.0,31.95,0,172.7,Emergency Department,Accident & Emergency,admit,Med-Surg ICU,0.000694,95.300000,NaN,122.0,703.03,0,0.0,NaN,NaN,NaN,3.0,6.0,0.0,5.0,NaN,102.0,NaN,0.0,68.0,37.0,NaN,36.7,NaN,0.0,NaN,NaN,NaN,88.0,48.0,88.0,48.0,96.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
3,81.0,22.64,1,165.1,Operating Room,Operating Room / Recovery,admit,CTICU,0.000694,61.700000,NaN,203.0,1206.03,1,0.0,NaN,NaN,NaN,4.0,6.0,0.0,5.0,185.0,114.0,25.9,1.0,60.0,4.0,NaN,34.8,NaN,1.0,8.0,62.0,30.0,48.0,42.0,48.0,42.0,116.0,...,11.6,8.9,34.0,25.9,1.6,1.1,NaN,NaN,198.0,43.0,5.0,3.5,NaN,NaN,9.0,8.0,NaN,NaN,1.6,1.1,37.0,27.0,7.44,7.34,337.0,102.0,342.5,236.666667,0.040000,0.030000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
4,19.0,30.05,0,188.0,NaN,Accident & Emergency,admit,Med-Surg ICU,0.073611,106.206601,NaN,119.0,601.01,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,0.0,103.0,16.0,NaN,36.7,NaN,0.0,NaN,NaN,NaN,99.0,57.0,99.0,57.0,89.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001222,0.009997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma


In [ ]:
cat_cols = ['hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem']

In [ ]:
for usecol in cat_cols:
    X[usecol] = X[usecol].astype('str')
    test_v1[usecol] = test_v1[usecol].astype('str')
    
    #Fit LabelEncoder
    le = LabelEncoder().fit(np.unique(X[usecol].unique().tolist() + test_v1[usecol].unique().tolist()))

    #At the end 0 will be used for dropped values
    X[usecol] = le.transform(X[usecol])+1
    test_v1[usecol]  = le.transform(test_v1[usecol])+1
    
    X[usecol] = X[usecol].replace(np.nan, '').astype('int').astype('category')
    test_v1[usecol]  = test_v1[usecol].replace(np.nan, '').astype('int').astype('category')

In [ ]:
# Threshold for removing correlated variables
threshold = 0.8

# Absolute value correlation matrix
corr_matrix = X.corr().abs()
corr_matrix.head()

,age,bmi,elective_surgery,height,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,...,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_glucose_max,h1_glucose_min,h1_inr_max,h1_inr_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
age,1.000000,0.092855,0.064233,0.115988,0.049128,0.134732,0.117329,0.021230,0.056682,0.056407,0.002495,0.045789,0.235112,0.055899,0.023706,0.023103,0.007708,0.020667,0.005970,0.145755,0.117991,0.014613,0.016189,0.038917,0.038287,0.082630,0.177496,0.041048,0.009531,0.187675,0.248703,0.061195,0.210014,0.058729,0.207810,0.131661,0.140483,0.026683,0.112225,0.006111,...,0.061097,0.096894,0.152484,0.144106,0.122238,0.119398,0.105155,0.115574,0.021891,0.043488,0.081118,0.080597,0.060127,0.109750,0.008083,0.038857,0.002146,0.000338,0.074605,0.059894,0.105155,0.115574,0.039537,0.054652,0.044722,0.054035,0.019082,0.043476,0.132533,0.118129,0.154213,0.081957,0.029653,0.028799,0.075040,0.020597,0.023880,0.030072,0.022666,0.025236
bmi,0.092855,1.000000,0.017858,0.042170,0.000946,0.891465,0.054109,0.027793,0.005211,0.017295,0.006689,0.000258,0.047521,0.067872,0.012949,0.021202,0.005171,0.028666,0.101896,0.022265,0.097768,0.037506,0.056725,0.001418,0.021386,0.039811,0.054900,0.070429,0.030751,0.065864,0.052001,0.054949,0.005778,0.055780,0.005845,0.031586,0.014002,0.036400,0.002536,0.062021,...,0.095705,0.091741,0.057876,0.061565,0.092228,0.092476,0.011213,0.016527,0.016113,0.015138,0.017866,0.017220,0.087086,0.093425,0.024899,0.016613,0.023861,0.022881,0.067438,0.075944,0.011213,0.016527,0.181233,0.182472,0.052099,0.059249,0.094733,0.108870,0.171231,0.169991,0.036867,0.015857,0.020162,0.002589,0.172104,0.001635,0.030734,0.013755,0.010075,0.042609
elective_surgery,0.064233,0.017858,1.000000,0.023439,0.133704,0.026881,0.024966,0.367040,0.775025,0.908247,0.027357,0.015510,0.156585,0.100575,0.009830,0.015783,0.012191,0.013365,0.027103,0.069853,0.119344,0.160831,0.003141,0.140857,0.046775,0.040545,0.022915,0.146526,0.078414,0.004486,0.012423,0.191706,0.007190,0.185851,0.017472,0.073569,0.025685,0.011269,0.036367,0.141750,...,0.009556,0.019669,0.055722,0.114783,0.071720,0.134437,0.112628,0.125849,0.006596,0.011856,0.055013,0.086435,0.098103,0.034988,0.056927,0.026902,0.098373,0.040979,0.098106,0.118581,0.112628,0.125849,0.008744,0.051061,0.074501,0.011893,0.182023,0.021262,0.076172,0.009047,0.093736,0.058480,0.006229,0.031512,0.001645,0.034700,0.014695,0.017587,0.008215,0.015369
height,0.115988,0.042170,0.023439,1.000000,0.008162,0.397350,0.061705,0.001734,0.015349,0.025106,0.010483,0.040522,0.019233,0.057794,0.008344,0.013447,0.001780,0.009559,0.006197,0.024560,0.126623,0.019587,0.033574,0.056493,0.024591,0.015115,0.080864,0.007005,0.015919,0.019729,0.071935,0.034316,0.136182,0.034649,0.136995,0.020729,0.028854,0.002377,0.020890,0.021677,...,0.027771,0.022254,0.164933,0.147421,0.142235,0.124921,0.001398,0.002758,0.005400,0.007998,0.086038,0.085625,0.053275,0.062376,0.022718,0.0234

In [ ]:
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()
# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
print('There are %d columns to remove.' % (len(to_drop)))
#Drop the columns with high correlations
X = X.drop(columns = to_drop)

There are 50 columns to remove.


In [ ]:
test_v1 = test_v1.drop(columns = to_drop)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=1)

# LGB Model building
d_train = lgb.Dataset(X_train, label=y_train)
params = {}
params['learning_rate'] = 0.1
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 100
params['min_data'] = 50
params['max_depth'] = 10

params_2 = {'bagging_fraction': 0.4,
 'boosting': 'dart',
 'colsample_bytree': 0.5048747931447324,
 'cat_smooth': 40, 
 'min_data_per_group': 600,
 'max_cat_threshold': 70,
 'learning_rate': 0.05,
 #'num_iterations': 10000,
 'max_bin': 1350,
 'max_depth': 14,
 'num_leaves': 4200,
 'min_child_samples': 410,
 'min_child_weight': 0.1,
 'min_data_in_leaf': 2420,
 'reg_alpha': 0.1,
 'reg_lambda': 20,
 'scale_pos_weight': 3,
 'subsample': 0.7340872997512691,
 'subsample_for_bin': 512,
 #'scoring': 'roc_auc',
 #'metric': 'auc',
 'metric': 'binary_logloss',
 'objective': 'binary'}

clf = lgb.train(params_2, d_train, 1800)

# Prediction
y_pred = np.round(clf.predict(X_val))

#Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred)
print(cm)

#Accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_val)
print(accuracy)

[[19840  1082]
 [  876  1131]]
0.9146059575210432


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_Scaled = scaler.fit_transform(X)
test_v1_scaled = scaler.fit_transform(test_v1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_Scaled, y, test_size=0.25, random_state=1)

In [ ]:
# Model building
d_train = lgb.Dataset(X_train, label=y_train)
params = {}
params['learning_rate'] = 0.05
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 4000
params['min_data'] = 50
params['max_depth'] = 12
clf = lgb.train(params, d_train, 10000)

params_2 = {'bagging_fraction': 0.4,
 'boosting': 'dart',
 'colsample_bytree': 0.5048747931447324,
 'cat_smooth': 40, 
 'min_data_per_group': 600,
 'max_cat_threshold': 70,
 'learning_rate': 0.05,
 'max_bin': 1312,
 'max_depth': 13,
 'num_leaves': 4100,
 'min_child_samples': 407,
 'min_child_weight': 0.1,
 'min_data_in_leaf': 2420,
 'reg_alpha': 0.1,
 'reg_lambda': 20,
 'scale_pos_weight': 3,
 'subsample': 0.7340872997512691,
 'subsample_for_bin': 512,
 'metric': 'binary_logloss',
 'objective': 'binary'}
clf = lgb.train(params_2, d_train, 2000)

#Prediction
y_pred = np.round(clf.predict(X_val))

#Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred)
print(cm)

#Accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_val)
print(accuracy)

[[19839  1083]
 [  874  1133]]
0.9146495704130141


In [ ]:
#Prediction on Test variables
pred_on_test = clf.predict(test_v1)

In [ ]:
solution = pd.read_csv(os.path.join(repo_path, 'data/solution_template.csv'))

In [ ]:
from google.colab import files ### remove this line of code if not using colab

solution.hospital_death = pred_on_test
solution.to_csv("lgbm_submission.csv", index=0)

!cp lgbm_submission.csv "/content/gdrive/My Drive/colab/Kaggle-WiDS/"
# files.download('lgbm_submission.csv') ### remove this line of code if not using colab